In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder

In [2]:
%pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [4]:
url_ligue1 = "https://fbref.com/en/comps/13/Ligue-1-Stats"
left_on = ["Time", "Comp", "Round", "Day", "Venue",	"Result", "GF","GA","Opponent"]
nb_saisons = 10


df_final = []

for i in range(nb_saisons):

    request1 = requests.get(url_ligue1)
    soup = BeautifulSoup(request1.text)
    url_equipes = soup.select("table.stats_table")[0].find_all("a")
    url_equipes = [equipes.get("href") for equipes in url_equipes]
    url_equipes = [equipes for equipes in url_equipes if equipes and "squads" in equipes]
    url_equipes = [f"https://fbref.com{i}" for i in url_equipes]
    url_ligue1 = f"https://fbref.com{soup.find('a', class_='button2 prev').get('href')}"


    
    for j in range(len(url_equipes)):

        request2 = requests.get(url_equipes[j])
        soup2 = BeautifulSoup(request2.text)
        df_equipe = pd.read_html(request2.text, match="Scores")[0]

        nom_equipe = url_equipes[j].split("/")[-1].replace("-Stats", "").replace("-", " ")

        df_equipe["equipe"] = nom_equipe



        url_stats = soup2.findAll("a")
        url_stats = [el.get("href") for el in url_stats]
        url_stats = [el for el in url_stats if el and "matchlogs/all_comps" in el]
        url_stats = [el for el in url_stats if el and ("passing/" in el or "shooting" in el)]
        url_stats = list(set(url_stats))
        url_stats = [f"https://fbref.com{i}" for i in url_stats]

        time.sleep(1)


        for y in range(len(url_stats)):

            request3 = requests.get(url_stats[y])
            soup3 = BeautifulSoup(request3.text)

            stats = pd.read_html(request3.text)[0]

            if stats.columns.nlevels > 1:
                stats.columns = [f"{col}_{branch}" if "For" not in col and "Unnamed:" not in col else f"{branch}" for col, branch in stats.columns]

            stats.drop(left_on + [col for col in stats.columns if 'Report' in col], axis=1, inplace=True)


            df_equipe = df_equipe.merge(stats, on="Date")
        
        df_final.append(df_equipe)
        time.sleep(1)


df_final2 = pd.concat(df_final)








        


        




    


    

In [7]:
df_final2['DateTime'] = pd.to_datetime(df_final2['Date'] + ' ' + df_final2['Time'])
df_final2 = df_final2.drop(["Date", "Time"], axis=1)
df_final2 = df_final2[['DateTime'] + [col for col in df_final2.columns if col != 'DateTime']]


KeyError: 'Date'

In [8]:
df_final2 = df_final2[df_final2["Comp"] == "Ligue 1"]

In [9]:
df_final2['Saison'] = df_final2['DateTime'].apply(lambda x: f"{x.year}-{x.year + 1}" if x.month >= 8 else f"{x.year - 1}-{x.year}")
df_final2

C:\Users\vtgra\AppData\Local\Temp\ipykernel_57796\217005003.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2['Saison'] = df_final2['DateTime'].apply(lambda x: f"{x.year}-{x.year + 1}" if x.month >= 8 else f"{x.year - 1}-{x.year}")


,DateTime,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,...,Standard_Dist,Standard_FK,Standard_PK,Standard_PKatt,Expected_xG,Expected_npxG,Expected_npxG/Sh,Expected_G-xG,Expected_np:G-xG,Saison
0,2023-08-12 21:00:00,Ligue 1,Matchweek 1,Sat,Home,D,0.0,0.0,Lorient,1.2,...,18.9,1.0,0,0,1.2,1.2,0.06,-1.2,-1.2,2023-2024
1,2023-08-19 21:00:00,Ligue 1,Matchweek 2,Sat,Away,D,1.0,1.0,Toulouse,2.1,...,15.9,0.0,1,1,2.1,1.3,0.09,-1.1,-1.3,2023-2024
2,2023-08-26 21:00:00,Ligue 1,Matchweek 3,Sat,Home,W,3.0,1.0,Lens,1.5,...,19.9,1.0,0,0,1.5,1.5,0.09,1.5,1.5,2023-2024
3,2023-09-03 20:45:00,Ligue 1,Matchweek 4,Sun,Away,W,4.0,1.0,Lyon,4.5,...,16.0,0.0,1,1,4.5,3.7,0.18,-0.5,-0.7,2023-2024
4,2023-09-15 21:00:00,Ligue 1,Matchweek 5,Fri,Home,L,2.0,3.0,Nice,1.0,...,20.6,1.0,0,0,1.0,1.0,0.08,1.0,1.0,2023-2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2015-04-26 17:00:00,Ligue 1,Matchweek 34,Sun,Home,L,0,3,Monaco,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2014-2015
34,2015-05-03 14:00:00,Ligue 1,Matchweek 35,Sun,Away,L,1,3,Lille,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2014-2015
35,2015-05-10 14:00:00,Ligue 1,Matchweek 36,Sun,Home,L,0,1,Montpellier,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2014-2015
36,2015-05-16 21:00:00,Ligue 1,Matchweek 37,Sat,Away,L,1,2,Nice,NaN,...,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN,2014-2015


In [10]:
df_final2.groupby('equipe')['Poss'].mean().reset_index().sort_values(by='Poss', ascending=False)
df_final2['Result'] = pd.Categorical(df_final2['Result'], categories=['W', 'D', 'L'])




C:\Users\vtgra\AppData\Local\Temp\ipykernel_57796\2642461274.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final2['Result'] = pd.Categorical(df_final2['Result'], categories=['W', 'D', 'L'])


In [38]:
df_final2.groupby("equipe")["Result"].value_counts().unstack()

Result,W,D,L
Formation,,,
3-1-4-2,12,15,21
3-2-4-1,6,2,1
3-3-3-1,2,0,1
3-4-1-2,96,79,96
3-4-3,179,140,166
3-4-3◆,2,1,6
3-5-1-1,21,17,21
3-5-2,64,58,85
4-1-2-1-2◆,118,63,77


In [44]:
df_final2["equipe"].nunique()

33

In [165]:
ML = df_final2.copy()

In [166]:
ML = ML.sort_values(by=['equipe', 'DateTime'])

In [167]:

ML['Formation'] = ML['Formation'].replace({'4-1-2-1-2◆': '4-1-2-1-2', '3-4-3◆': '3-4-3'}, regex=True)
ML["5_SoT_mean"] = ML.groupby('equipe')['Standard_SoT'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["2_SoT_mean"] = ML.groupby('equipe')['Standard_SoT'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)
ML["5_Total_Cmp_mean"] =ML.groupby('equipe')['Total_Cmp'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["2_Total_Cmp_mean"] =ML.groupby('equipe')['Total_Cmp'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)
ML["5_poss_mean"] =ML.groupby('equipe')['Poss'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["2_poss_mean"] =ML.groupby('equipe')['Poss'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)
ML["5_Standard_SoT"] =ML.groupby('equipe')['Standard_SoT'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["2_Standard_SoT"] =ML.groupby('equipe')['Standard_SoT'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)
ML["5_GF"] =ML.groupby('equipe')['GF'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["5_GA"] =ML.groupby('equipe')['GA'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["2_GF"] =ML.groupby('equipe')['GF'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)
ML["2_GA"] =ML.groupby('equipe')['GA'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)
ML["5_DistP"] =ML.groupby('equipe')['Total_TotDist'].shift().rolling(window=5, min_periods=5).mean().reset_index(level=0, drop=True)
ML["2_DistP"] =ML.groupby('equipe')['Total_TotDist'].shift().rolling(window=2, min_periods=2).mean().reset_index(level=0, drop=True)



In [ ]:
ML.groupby("equipe")

In [168]:
ML2 = ML.copy()
ML2 = ML2[["DateTime", "Saison", "equipe", "Venue", "Result", "Opponent", "Formation",  "5_SoT_mean", "2_SoT_mean", "5_Total_Cmp_mean", "2_Total_Cmp_mean", "5_poss_mean" , "2_poss_mean","5_Standard_SoT", "2_Standard_SoT", "5_GF", "5_GA", "2_GF", "2_GA", "5_DistP","2_DistP"]].dropna()
ML2 = ML2[ML2["Saison"] != "2023-2024"]
ML2['Formation'] = ML2['Formation'].astype('category')
ML22 = pd.get_dummies(ML2['Formation'], prefix='F_')
ML33 = pd.get_dummies(ML2['Opponent'], prefix='opp_')
ML44 = pd.get_dummies(ML2['Venue'], prefix='V_')

ML2 = pd.concat([ML2, ML22], axis=1)
ML2 = pd.concat([ML2, ML33], axis=1)
ML2 = pd.concat([ML2, ML44], axis=1)

ML2


,DateTime,Saison,equipe,Venue,Result,Opponent,Formation,5_SoT_mean,2_SoT_mean,5_Total_Cmp_mean,2_Total_Cmp_mean,5_poss_mean,2_poss_mean,5_Standard_SoT,2_Standard_SoT,5_GF,5_GA,2_GF,2_GA,5_DistP,2_DistP,F__3-1-4-2,F__3-2-4-1,F__3-3-3-1,F__3-4-1-2,F__3-4-3,F__3-5-1-1,F__3-5-2,F__4-1-2-1-2,F__4-1-3-2,F__4-1-4-1,F__4-2-2-2,F__4-2-3-1,F__4-3-1-2,F__4-3-2-1,F__4-3-3,F__4-4-1-1,F__4-4-2,F__4-5-1,F__5-3-2,F__5-4-1,opp__Ajaccio,opp__Amiens,opp__Angers,opp__Auxerre,opp__Bastia,opp__Bordeaux,opp__Brest,opp__Caen,opp__Clermont Foot,opp__Dijon,opp__Evian,opp__Gazélec Ajaccio,opp__Guingamp,opp__Lens,opp__Lille,opp__Lorient,opp__Lyon,opp__Marseille,opp__Metz,opp__Monaco,opp__Montpellier,opp__Nancy,opp__Nantes,opp__Nice,opp__Nîmes,opp__Paris S-G,opp__Reims,opp__Rennes,opp__Saint-Étienne,opp__Strasbourg,opp__Toulouse,opp__Troyes,V__Away,V__Home
5,2022-09-04 15:00:00,2022-2023,Ajaccio,Home,L,Lorient,4-4-2,1.8,1.0,295.2,359.0,43.6,52.0,1.8,1.0,0.6,1.8,0.5,2.5,5483.2,6789.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
6,2022-09-11 15:00:00,2022-2023,Ajaccio,Home,L,Nice,4-4-2,1.8,1.5,324.6,414.5,48.6,60.0,1.8,1.5,0.4,1.6,0.0,1.5,6173.6,7978.5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
7,2022-09-18 15:00:00,2022-2023,Ajaccio,Away,W,Brest,4-4-2,2.0,3.0,342.6,349.0,50.4,51.5,2.0,3.0,0.4,1.8,0.0,1.0,6445.0,6670.5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
8,2022-10-02 15:00:00,2022-2023,Ajaccio,Home,L,Clermont Foot,4-4-2,2.4,3.5,333.2,281.0,49.6,43.0,2.4,3.5,0.4,1.4,0.5,0.5,6278.8,5096.0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True
9,2022-10-08 17:00:00,2022-2023,Ajaccio,Away,W,Marseille,4-4-2,2.6,3.5,354.2,315.0,51.6,46.5,2.6,3.5,0.4,1.4,1.0,1.5,6587.8,5632.5,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33,2023-04-30 15:00:00,2022-2023,Troyes,Home,L,Nice,3-4-3,1.4,1.0,295.0,245.0,43.2,34.5,1.4,1.0,0.4,2.2,0.0,1.5,5410.2,4422.5,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True
34,2023-05-07 20:45:00,2022-2023,Troyes,Home,L,Paris S-G,3-4-3,1.4,1.0,314.0,242.5,44.4,32.0,1.4,1.0,0.2,1.8,0.0,1.5,5635.0,4127.5,False,False,False,False,T

In [172]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report



cutoff_date = datetime(2022, 9, 1)

X_train = ML2[ML2["DateTime"] <= cutoff_date].drop(["Result", "Opponent","Venue", "DateTime", "equipe", "Saison", "Formation"], axis=1)
y_train = ML2[ML2["DateTime"] <= cutoff_date]["Result"]

X_test = ML2[ML2["DateTime"] > cutoff_date].drop(["Result","Opponent", "Venue", "DateTime", "equipe", "Saison", 'Formation'], axis=1)
y_test = ML2[ML2["DateTime"] > cutoff_date]["Result"]



random_forest_model = RandomForestClassifier(n_estimators=1000, random_state=5)
random_forest_model.fit(X_train, y_train)


y_pred = random_forest_model.predict(X_test)

# Évaluez les performances du modèle
accuracy = accuracy_score(y_test, y_pred)
classification_report_result = classification_report(y_test, y_pred)


print(f"Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report_result)


Accuracy: 0.4084
Classification Report:
               precision    recall  f1-score   support

           D       0.27      0.23      0.25       142
           L       0.44      0.50      0.47       229
           W       0.45      0.42      0.44       224

    accuracy                           0.41       595
   macro avg       0.39      0.39      0.39       595
weighted avg       0.40      0.41      0.40       595

